## Job Web-Scraping Using Selenium
Note: This is intented for practice to extract real time job descriptions. Please adhere to the Robots.txt

In [19]:
#If you are using Selenium for the first time, please download the webdriver and note the filepath
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

import pandas as pd
import numpy as np

#Used to allow the browser to load. 
import time
from tqdm import tqdm

#used as a import for dateing the CSV
import datetime

import warnings
warnings.filterwarnings("ignore")

In [20]:
#this is an input for jobs and location
occupation = str(input("Job title: "))
loc = str(input("City, State(i.e. TX): "))

Job title: Data Scientist
City, State(i.e. TX): Atlanta, GA


### You will need to folder that the webdriver is in:

Download the webdriver here: https://www.seleniumhq.org/download/

Note the path you download the file from. Locate the file and unzip it.

In [35]:
path_to_chromedriver = "./chromedriver_win32/chromedriver.exe"

In [36]:
#setup browser window
browser = webdriver.Chrome(executable_path= path_to_chromedriver)

In [37]:
#adds the data in URL format by removing any white space and adding the data. 
def Keyword_to_url(kw):
    kw = kw.strip(" ").replace(" ","%20")
    return kw

In [38]:
# this will be used as the URL input for multiple job searches. 
radius = str("5")
page_size = "100" #how many pages would you like to get. 
source = 'IN' #stands for indeed

In [39]:
#give it the siteD
url = 'https://www.careeronestop.org/Toolkit/Jobs/find-jobs.aspx?keyword='+ \
    Keyword_to_url(occupation)+\
    '&ajax=0&location='+loc+'&radius='+radius+ \
    '&source='+ source +'&pagesize='+page_size+'&sortcolumns=accquisitiondate&sortdirections=DSC'
browser.get(url)
time.sleep(1)

In [40]:
listing_num = len(browser.find_elements_by_xpath('//*[@data-title="Job Title"]'))

In [41]:
job_links = browser.find_elements_by_xpath('//*[@data-title="Job Title"]')

In [42]:
#these catch the title of the job details. 
company = []
position = []
location = []
date_posted = []
job_url = []
pos_description = []

In [43]:
def collect_job_listings(num):
    
    """
    Works on an individual link basis.
    Creates dataframe from the link and job listed on the site. 
    the job descriptions will be added later.
    """
    
    num = int(num)
    position.append(job_links[num].text)
    company.append(browser.find_elements_by_xpath('//*[@data-title="Company"]')[num].text)
    location.append(browser.find_elements_by_xpath('//*[@data-title="Location"]')[num].text)
    date_posted.append(browser.find_elements_by_xpath('//*[@data-title="Date Posted"]')[num].text)
    job_url.append(job_links[num].find_element_by_tag_name("a").get_attribute('href'))
    
    return pd.DataFrame({'company': company , 
                     'position': position, 
                     'location':location,
                     'date_posted':date_posted,'job_url': job_url,
                     'position_description':'0'},
                        columns=['company','location','position','date_posted','job_url','position_description'])
    

In [44]:
#loop to obtain all information for each job in the table.
for links in range(listing_num):
    collect_job_listings(links)

In [45]:
#creating a dataframe of all data collected.
job_list = pd.DataFrame({'company': company , 
                     'position': position, 
                     'location':location,
                     'date_posted':date_posted,'job_url': job_url,
                     'position_description':'0'},
                        columns=['company','location','position','date_posted','job_url','position_description']).drop_duplicates()
#drops all duplicates and replaces them with

In [46]:
job_list

,company,location,position,date_posted,job_url,position_description
0,Softcrylic LLC,"Atlanta, GA",Senior Data Scientist,07/03/2018,http://www.indeed.com/viewjob?jk=1570a9c17b86f...,0
1,Travelport,"Atlanta, GA",Senior Data Scientist II,07/02/2018,http://www.indeed.com/viewjob?jk=2b281b748d3e4...,0
2,Scientific Research Corporation,"Atlanta, GA",Compensation and HRIS Analyst,07/02/2018,http://www.indeed.com/viewjob?jk=4db989e56cc13...,0
3,SunTrust,"Atlanta, GA",Data & Analytics Developer,07/02/2018,http://www.indeed.com/viewjob?jk=c7f471aecde80...,0
4,"IHRC, Inc.","Atlanta, GA",Laboratory Data Manager I,07/02/2018,http://www.indeed.com/viewjob?jk=2c04e540ffc49...,0
5,The Home Depot,"Atlanta, GA",DATA SCIENTIST,07/01/2018,http://www.indeed.com/viewjob?jk=240b3dd6b3d53...,0
6,Equifax,"Atlanta, GA",Data Scientist,06/30/2018,http://www.indeed.com/viewjob?jk=553b92cee1ba4...,0
7,Wells Fargo,"Atlanta, GA",Data Scientist / Analytic Consultant 3,06/29/2018,http://www.indeed.com/viewjob?jk=0faa4ca725023...,0
8,Georgia Tech Research Institute,"Atlanta, GA",Junior Software Developer - ISD,06/29/2018,http://www.indeed.com/viewjob?jk=39f64f5613cdc...,0
9,McKinsey & Company,"Atlanta, GA","Senior Analyst - GC Insights, New Ventures",06/29/2018,http://www.indeed.com/viewjob?jk=02c4500160f53...,0


In [14]:
#checks for page errors and to see which ones. 
err = []

In [15]:
#Loops the urls and receives the content from the webpage.
for i in tqdm(range(job_list.shape[0])):
    time.sleep(np.random.randint(2,7))
    try:
        browser.get(job_list['job_url'][i])
        pos_des = browser.find_element_by_class_name("snip").text
        pos_des = pos_des.replace("\n",' ')
        job_list['position_description'][i] = job_list['position_description'][i].replace('0',pos_des)
    except:
        err.append(i)
        

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [08:38<00:00,  5.18s/it]


In [16]:
browser.get(job_list['job_url'][1])

In [17]:
browser.close()

In [18]:
job_list.to_csv("./data/"+str(datetime.datetime.today())[:10]+"_"+occupation+"_"+loc+" job_list.csv")